Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10


def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [69]:
image_size = 28 * 28
label_size = 10


class BaseNN(object):
    def __init__(self, hidden_layer_sizes, sgd_batch_size, alpha, num_steps):
        self.layer_sizes = [image_size] + hidden_layer_sizes + [label_size]
        self.sgd_batch_size = sgd_batch_size
        self.alpha = alpha
        self.num_steps = num_steps

        self.graph = tf.Graph()
        with self.graph.as_default():
            self.tf_train_dataset = tf.placeholder(tf.float32, shape=(sgd_batch_size, image_size))
            self.tf_train_labels = tf.placeholder(tf.float32, shape=(sgd_batch_size, label_size))
            self.tf_valid_dataset = tf.constant(valid_dataset)
            self.tf_test_dataset = tf.constant(test_dataset)

            self.weights = [
                tf.Variable(tf.truncated_normal([self.layer_sizes[i], self.layer_sizes[i + 1]]))
                for i in range(len(self.layer_sizes) - 1)]
            self.biases = [
                tf.Variable(tf.zeros([self.layer_sizes[i + 1]]))
                for i in range(len(self.layer_sizes) - 1)]

            logits = self.forward_prop_train(self.tf_train_dataset)
            self.loss = self.calculate_loss(logits, self.tf_train_labels)
            self.optimizer = tf.train.GradientDescentOptimizer(alpha).minimize(self.loss)

            self.train_prediction = tf.nn.softmax(logits)
            self.valid_prediction = tf.nn.softmax(self.forward_prop(self.tf_valid_dataset))
            self.test_prediction = tf.nn.softmax(self.forward_prop(self.tf_test_dataset))

    def forward_prop(self, tf_dataset):
        res = tf_dataset
        for i in range(len(self.weights)):
            res = tf.nn.relu(res) if i > 0 else res
            res = tf.matmul(res, self.weights[i]) + self.biases[i]
        return res

    def forward_prop_train(self, tf_dataset):
        return self.forward_prop(tf_dataset)

    def calculate_loss(self, logits, labels):
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))

    def fit(self, train_dataset, train_labels, log_every=500):
        with tf.Session(graph=self.graph) as session:
            tf.initialize_all_variables().run()
            print('Initialized')

            for step in range(self.num_steps):
                random_indices = np.random.choice(
                    train_dataset.shape[0], self.sgd_batch_size)
                batch_data = train_dataset[random_indices]
                batch_labels = train_labels[random_indices]

                _, l, predictions = session.run(
                    [self.optimizer, self.loss, self.train_prediction],
                    feed_dict={
                        self.tf_train_dataset: batch_data,
                        self.tf_train_labels: batch_labels})

                if (step % log_every == 0):
                    print("Minibatch loss at step %d: %f" % (step, l))
                    print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                    print("Validation accuracy: %.1f%%" % accuracy(self.valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(self.test_prediction.eval(), test_labels))


class L2LossNN(BaseNN):
    def __init__(self, *args, **kwargs):
        self.beta = kwargs.pop('beta')
        super(L2LossNN, self).__init__(*args, **kwargs)
    
    def calculate_loss(self, logits, labels):
        return tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, labels) +
            self.beta * tf.add_n([tf.nn.l2_loss(w) for w in self.weights + self.biases]))
    

In [52]:
L2LossNN(
    hidden_layer_sizes=[256], 
    sgd_batch_size=128,
    alpha=0.5, 
    beta=0.001, 
    num_steps=10001
).fit(train_dataset, train_labels)

Initialized
Minibatch loss at step 0: 288.164062
Minibatch accuracy: 6.2%
Validation accuracy: 32.2%
Minibatch loss at step 500: 50.046463
Minibatch accuracy: 73.4%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 29.446426
Minibatch accuracy: 75.0%
Validation accuracy: 79.6%
Minibatch loss at step 1500: 17.567398
Minibatch accuracy: 85.9%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 10.677494
Minibatch accuracy: 87.5%
Validation accuracy: 83.6%
Minibatch loss at step 2500: 6.784433
Minibatch accuracy: 81.2%
Validation accuracy: 84.0%
Minibatch loss at step 3000: 4.206826
Minibatch accuracy: 84.4%
Validation accuracy: 85.8%
Minibatch loss at step 3500: 2.705393
Minibatch accuracy: 89.8%
Validation accuracy: 86.4%
Minibatch loss at step 4000: 1.865380
Minibatch accuracy: 85.2%
Validation accuracy: 87.3%
Minibatch loss at step 4500: 1.306390
Minibatch accuracy: 88.3%
Validation accuracy: 87.5%
Minibatch loss at step 5000: 0.884554
Minibatch accuracy: 90.6%
Validation ac

In [70]:
L2LossNN(
    hidden_layer_sizes=[256, 64], 
    sgd_batch_size=128, 
    alpha=0.1, 
    beta=0.001, 
    num_steps=30001
).fit(train_dataset, train_labels, log_every=1500)

Initialized
Minibatch loss at step 0: 774.489807
Minibatch accuracy: 11.7%
Validation accuracy: 23.1%
Minibatch loss at step 1500: 64.914047
Minibatch accuracy: 33.6%
Validation accuracy: 35.0%
Minibatch loss at step 3000: 47.833633
Minibatch accuracy: 57.8%
Validation accuracy: 54.8%
Minibatch loss at step 4500: 35.786533
Minibatch accuracy: 61.7%
Validation accuracy: 61.3%
Minibatch loss at step 6000: 26.739397
Minibatch accuracy: 61.7%
Validation accuracy: 67.1%
Minibatch loss at step 7500: 19.662354
Minibatch accuracy: 74.2%
Validation accuracy: 74.3%
Minibatch loss at step 9000: 14.706854
Minibatch accuracy: 78.9%
Validation accuracy: 79.1%
Minibatch loss at step 10500: 10.824858
Minibatch accuracy: 86.7%
Validation accuracy: 82.2%
Minibatch loss at step 12000: 8.275635
Minibatch accuracy: 82.0%
Validation accuracy: 83.9%
Minibatch loss at step 13500: 6.082581
Minibatch accuracy: 87.5%
Validation accuracy: 85.1%
Minibatch loss at step 15000: 4.679677
Minibatch accuracy: 86.7%
Vali

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [58]:
# No regularization with only 3000 samples; heavily overfitted

L2LossNN(
    hidden_layer_sizes=[256], 
    sgd_batch_size=128,
    alpha=0.5, 
    beta=0, 
    num_steps=2001
).fit(train_dataset[:3000], train_labels[:3000], log_every=200)

Initialized
Minibatch loss at step 0: 161.214417
Minibatch accuracy: 7.8%
Validation accuracy: 34.4%
Minibatch loss at step 200: 0.399056
Minibatch accuracy: 97.7%
Validation accuracy: 76.4%
Minibatch loss at step 400: 0.002355
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Minibatch loss at step 600: 3.052499
Minibatch accuracy: 96.1%
Validation accuracy: 76.9%
Minibatch loss at step 800: 0.004029
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 1000: 0.000009
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1200: 0.106549
Minibatch accuracy: 99.2%
Validation accuracy: 77.2%
Minibatch loss at step 1400: 2.994856
Minibatch accuracy: 99.2%
Validation accuracy: 77.1%
Minibatch loss at step 1600: 0.000206
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 1800: 0.000001
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 2000: 0.000053
Minibatch accuracy: 100.0%
Validation acc

In [60]:
# With some regularization; less overfitted than above one

L2LossNN(
    hidden_layer_sizes=[256], 
    sgd_batch_size=128,
    alpha=0.5, 
    beta=0.01,
    num_steps=5001
).fit(train_dataset[:3000], train_labels[:3000])

Initialized
Minibatch loss at step 0: 949.289307
Minibatch accuracy: 9.4%
Validation accuracy: 31.1%
Minibatch loss at step 500: 5.648594
Minibatch accuracy: 98.4%
Validation accuracy: 81.9%
Minibatch loss at step 1000: 0.618355
Minibatch accuracy: 92.2%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 0.567008
Minibatch accuracy: 95.3%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 0.618557
Minibatch accuracy: 92.2%
Validation accuracy: 82.1%
Minibatch loss at step 2500: 0.574018
Minibatch accuracy: 96.1%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.535748
Minibatch accuracy: 95.3%
Validation accuracy: 82.6%
Minibatch loss at step 3500: 0.563654
Minibatch accuracy: 95.3%
Validation accuracy: 81.6%
Minibatch loss at step 4000: 0.621553
Minibatch accuracy: 95.3%
Validation accuracy: 82.4%
Minibatch loss at step 4500: 0.577981
Minibatch accuracy: 94.5%
Validation accuracy: 80.8%
Minibatch loss at step 5000: 0.559099
Minibatch accuracy: 93.8%
Validation accura

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [76]:
class DropoutNN(BaseNN):
    def __init__(self, *args, **kwargs):
        self.keep_prob = kwargs.pop('keep_prob')
        super(DropoutNN, self).__init__(*args, **kwargs)

    def forward_prop_train(self, tf_dataset):
        res = tf_dataset
        for i in range(len(self.weights)):
            res = tf.nn.dropout(tf.nn.relu(res), self.keep_prob) if i > 0 else res
            res = tf.matmul(res, self.weights[i]) + self.biases[i]
        return res
            

In [78]:
DropoutNN(
    hidden_layer_sizes=[256], 
    sgd_batch_size=128,
    alpha=0.5, 
    keep_prob=0.5,
    num_steps=5001
).fit(train_dataset[:3000], train_labels[:3000])

Initialized
Minibatch loss at step 0: 234.625916
Minibatch accuracy: 7.8%
Validation accuracy: 31.4%
Minibatch loss at step 500: 2.816261
Minibatch accuracy: 82.8%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 1.083063
Minibatch accuracy: 88.3%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 3.001185
Minibatch accuracy: 93.8%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 0.058738
Minibatch accuracy: 96.9%
Validation accuracy: 79.0%
Minibatch loss at step 2500: 0.336682
Minibatch accuracy: 93.8%
Validation accuracy: 79.7%
Minibatch loss at step 3000: 0.080011
Minibatch accuracy: 96.9%
Validation accuracy: 79.4%
Minibatch loss at step 3500: 0.532647
Minibatch accuracy: 96.9%
Validation accuracy: 79.2%
Minibatch loss at step 4000: 0.484350
Minibatch accuracy: 96.1%
Validation accuracy: 79.8%
Minibatch loss at step 4500: 0.109667
Minibatch accuracy: 96.9%
Validation accuracy: 80.0%
Minibatch loss at step 5000: 0.140859
Minibatch accuracy: 95.3%
Validation accura

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
DropoutNN(
    hidden_layer_sizes=[256, 64], 
    sgd_batch_size=128,
    alpha=0.1, 
    keep_prob=0.5,
    num_steps=5001
).fit(train_dataset[:3000], train_labels[:3000])